In [1]:
import numpy as np
import pandas as pd
import requests
from time import time
import folium
import os

In [126]:
filenames = os.listdir("./data")
data_all = pd.DataFrame()
for filename in filenames:
    if 'summary' not in filename:
        df = pd.read_pickle(os.path.join('data', filename))
        data_all = pd.concat([data_all, df])
    
data_all = data_all.astype({'date': 'datetime64'}).drop_duplicates()
data_all.to_pickle(f"data/parking_summary_{str(int(time()))}.pickle")

In [2]:
url = "https://data.corkcity.ie/api/3/action/datastore_search?resource_id=6fa2e95d-90a4-4a96-bf30-5d4756f6a777&limit=1000"
r = requests.get(url)
data0 = pd.DataFrame.from_dict(dict(r.json())['result']['records'])
data0

,_id,OBJECTID,Name,Address,Website,Type,MUGA,Longitude,Latitude
0,1,1,Atlantic Pond,"Ballintemple, Cork",https://www.corkcity.ie/corkcityco/en/things-t...,Walk,,-8.430833,51.900248
1,2,2,Blackrock Amenity Walk,"Castle Road, Blackrock, Cork",https://www.corkcity.ie/corkcityco/en/things-t...,Walk,,-8.402421,51.900298
2,3,3,South Parish Walk,"Queens Old Castle, Grand Parade, Cork",https://www.corkcity.ie/corkcityco/en/things-t...,Walk,,-8.475779,51.898477
3,4,4,Central Island Walk,"Daunt Square, Cork",https://www.corkcity.ie/corkcityco/en/things-t...,Walk,,-8.475857,51.898563
4,5,5,Shandon Walk,"Shandon, Cork",https://www.corkcity.ie/corkcityco/en/things-t...,Walk,,-8.476350,51.902808
...,...,...,...,...,...,...,...,...,...
97,98,98,The Mardyke Skate Park,,,Park,,-8.490278,51.897506
98,99,99,Tramore Valley Park BMX Track,,,MUGA,,-8.459941,51.880290
99,100,100,Douglas Community Park Multi Use Game Area,,,MUGA,,-8.438161,51.874946
100,101,101,Tory Top Park Multi Use Game Area,,,MUGA,,-8.474019,51.885372


In [100]:
url = 'https://data.corkcity.ie/api/3/action/datastore_search?resource_id=f4677dac-bb30-412e-95a8-d3c22134e3c0&limit=500'

r = requests.get(url)
data0 = pd.DataFrame.from_dict(dict(r.json())['result']['records']).astype({'date': 'datetime64'})
data0.to_pickle(f"data/parking_{str(int(time()))}.pickle")
data0

,_id,identifier,name,spaces,free_spaces,opening_times,height_restrictions,price,notes,latitude,longitude,date
0,1,1,Paul Street,749,273,"Monday - Saturday 07.30 -00.00, Sunday 11.30 -...",2m,€2.30 per hour; Flat rate €3.50 from 18.30-24.00,,51.900542,-8.475415,2022-05-15 16:40:03
1,2,2,North Main Street,330,244,"Monday - Saturday 07.30 - 21.30, Sunday 11.30 ...",2m,€1.70 per hour; Flat rate €2.00 from 18.30-21.30,,51.901008,-8.477804,2022-05-15 16:40:03
2,7,104,Saint Finbarr's,350,330,24/7,2m,€2.80 per hour,,51.896723,-8.482056,2022-05-14 21:49:02
3,3,9,Black Ash Park & Ride,935,817,Monday - Saturday 06:45 - 20:00,2.1m,€5 per day,,51.878279,-8.466956,2022-05-15 16:45:00
4,4,108,Merchants Quay,710,530,"Monday - Wednesday 08:00 -18:00, Thursday - Sa...",1.98m,€3 per hour,,51.899577,-8.468648,2022-05-15 16:40:26
5,5,103,City Hall - Eglington Street,436,108,24/7,2m,"€2.90 per hour, max. €13 per day",,51.896579,-8.464302,2022-05-15 15:46:02
6,6,102,Carrolls Quay,376,183,24/7,2m,€2.80 per hour,,51.901788,-8.472013,2022-05-15 15:46:02


In [102]:
df = data0.copy()

In [95]:
m = folium.Map(location=df[['latitude', 'longitude']].mean().tolist(), zoom_start=13, width=800, height=400)

for idx, row in df.iterrows():
    popup_html = f"""
    <b> {row['name']} </b> <br>
    Spaces: {row['spaces']}
    """
    folium.Marker(row[['latitude', 'longitude']].values.tolist(),
                  popup=folium.Popup(popup_html, max_width=500)
                 ).add_to(m)

m